In [15]:
# Function to calculate tree parameters (u, d, p) for different models
def calculate_tree_parameters(r, q, sigma, dt, model_type, drift=0, AmeEur='a'):
    if model_type == 'CRR':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(r * dt) - d) / (u - d)
    elif model_type == 'CRR_Drift':
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp((r + drift) * dt) - d) / (u - d)
    elif model_type == 'Jarrow-Rudd':
        u = np.exp(((b) - sigma**2 / 2) * dt + sigma * np.sqrt(dt))
        d = np.exp(((b) - sigma**2 / 2) * dt - sigma * np.sqrt(dt))
        p = (np.exp((b) * dt) - d) / (u - d)
    elif model_type == 'Tian':
        eta=np.exp(dt*sigma**2)
        u=0.5*np.exp(r*dt)*eta*(eta+1+np.sqrt(eta**2+2*eta-3))
        d=0.5*np.exp(r*dt)*eta*(eta+1-np.sqrt(eta**2+2*eta-3))
        p=(np.exp((b)*dt)-d)/(u-d)
    elif model_type == 'EQP':
        p=0.5
        u=np.exp(((b)-sigma**2/2)*dt+sigma*np.sqrt(dt))
        d=np.exp(((b)-sigma**2/2)*dt-sigma*np.sqrt(dt))
    elif model_type == 'Haahtela':
        u = np.exp(r * dt) * (1 + np.sqrt(np.exp((sigma ** 2) * dt) - 1))
        d = np.exp(r * dt) * (1 - np.sqrt(np.exp((sigma ** 2) * dt) - 1))
        p = (np.exp((r - q) * dt) - d) / (u - d)
    else:
        raise ValueError("Unknown model type provided.")

    return u, d, p

In [19]:

def calculate_bond_option_value(AmeEur='a', S=75, T2=5, t1=0, r=0.07, q=0, k=0.03, sigma=0.2, F=100, X=100, Coupon=6, N=5 ,model_type = 'CRR', drift = 0):
    # Parametri derivati
    b = r - q
    CRatio = F / X
    OptionValue = np.zeros(N + 1)
    ConversionProbability = np.zeros(N + 1)
    dt = T2 / N
    u, d, p = calculate_tree_parameters(r, q, sigma, dt, model_type, drift, AmeEur='a')
    # Calcolo dei fattori up e down e della probabilità
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r - q) * dt) - d) / (u - d)

    # Calcolo del valore iniziale dell'opzione e della probabilità di conversione
    for i in range(N + 1):
        StockPrice = S * (u ** i) * (d ** (N - i))
        OptionValue[i] = np.max([F + Coupon, CRatio * StockPrice])
        if OptionValue[i] == F + Coupon:
            ConversionProbability[i] = 0
        else:
            ConversionProbability[i] = 1

    CouponTime = T2

    # Induzione all'indietro per gli intervalli temporali precedenti
    for j in range(N, 0, -1):
        if CouponTime != (T2 - np.ceil((N - j + 1) * dt)):
            CouponPayment = 1
            CouponTime = T2 - np.ceil((N - j) * dt)
            Time = CouponTime - dt * j
        else:
            CouponPayment = 0
        for i in range(j):
            ru = ConversionProbability[i + 1] * r + (1 - ConversionProbability[i + 1]) * (r + k)
            rd = ConversionProbability[i] * r + (1 - ConversionProbability[i]) * (r + k)
            StockPrice = S * (u ** i) * (d ** (np.abs(i - j)))

            if CouponPayment == 1:
                CouponValue = Coupon * (p * np.exp(-ru * Time) + (1 - p) * np.exp(-rd * Time))
            else:
                CouponValue = 0

            if AmeEur == 'e' or (AmeEur == 'a' and dt * j <= t1):
                OptionValue[i] = CouponValue + p * OptionValue[i + 1] * np.exp(-ru * dt) + (1 - p) * OptionValue[i] * np.exp(-rd * dt)
                ConversionProbability[i] = p * ConversionProbability[i + 1] + (1 - p) * ConversionProbability[i]
            elif AmeEur == 'a':
                OptionValue[i] = np.max([CRatio * (StockPrice - X) + F, CouponValue + p * OptionValue[i + 1] * np.exp(-ru * dt) + (1 - p) * OptionValue[i] * np.exp(-rd * dt)])
                print(CRatio * (StockPrice - X) + F, CouponValue + p * OptionValue[i + 1] * np.exp(-ru * dt) + (1 - p) * OptionValue[i] * np.exp(-rd * dt))
                if OptionValue[i] == CRatio * (StockPrice - X) + F:
                    ConversionProbability[i] = 1
                else:
                    ConversionProbability[i] = p * ConversionProbability[i + 1] + (1 - p) * ConversionProbability[i]

    # Restituisci il valore dell'opzione al tempo 0
    return OptionValue[0]

# Chiamata di esempio della funzione con i parametri predefiniti
result = calculate_bond_option_value()
print("Valore dell'opzione al tempo 0:", result)


27.59095808785817 100.54526590616258
41.16087270705197 100.54526590616258
61.404806480848634 100.54526590616258
91.60520686201274 127.04581573172149
136.6589100292882 185.41570218787658
33.699672308791605 96.97711879827114
50.27400345267294 96.97711879827114
74.99999999999999 113.85814073710361
111.88685232309528 158.45730010905237
41.16087270705198 93.74852578199418
61.40480648084863 104.38647032415977
91.60520686201274 137.5642019390279
50.27400345267294 97.48353038207992
75.0 121.54001277418502
61.404806480848634 109.39678013503845
Valore dell'opzione al tempo 0: 106.23456548322234
